## Start

In [1]:
import os
import pickle
import time

import regex as re
import numpy as np

from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from typing import BinaryIO
from collections import defaultdict
from collections.abc import Iterable, Iterator

In [2]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
TOKEN = tuple[bytes]
PAIR = tuple[bytes, bytes]
END_TOKEN = '<|endoftext|>'

## 1 Assignment Overview

## 2 Byte-Pair Encoding (BPE) Tokenizer

### 2.1 The Unicode Standard

In [3]:
[ord('牛'), chr(29275)]

[29275, '牛']

#### Problem (unicode1)

##### a

In [ ]:
chr(0)

'\x00'

##### b

In [ ]:
print(chr(0))

 


##### c

In [8]:
"this is a test" + chr(0) + "string"

'this is a test\x00string'

In [9]:
print("this is a test" + chr(0) + "string")

this is a test string


### 2.2 Unicode Encodings

In [10]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'


In [11]:
print(type(utf8_encoded))

<class 'bytes'>


In [12]:
list(utf8_encoded)

[104,
 101,
 108,
 108,
 111,
 33,
 32,
 227,
 129,
 147,
 227,
 130,
 147,
 227,
 129,
 171,
 227,
 129,
 161,
 227,
 129,
 175,
 33]

In [13]:
[len(test_string),len(utf8_encoded)]

[13, 23]

In [14]:
print(utf8_encoded.decode("utf-8"))

hello! こんにちは!


#### Problem (unicode2)

##### a

##### b

In [15]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])
decode_utf8_bytes_to_str_wrong("hello".encode("utf-8"))

'hello'

In [16]:
decode_utf8_bytes_to_str_wrong("hello, 你好".encode("utf-8"))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 0: unexpected end of data

##### c

In [18]:
sentence = "hello, 你好".encode("utf-8")
[bytes([b]) for b in sentence]

[b'h',
 b'e',
 b'l',
 b'l',
 b'o',
 b',',
 b' ',
 b'\xe4',
 b'\xbd',
 b'\xa0',
 b'\xe5',
 b'\xa5',
 b'\xbd']

In [24]:
(b'\xe4\xbd\xa0').decode("utf-8")

'你'

### 2.3 Subword Tokenization

### 2.4 BPE Tokenizer Training

In [27]:
re.findall(PAT, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

In [28]:
max([("A", "B"), ("A", "C"), ("B", "ZZ"), ("BA", "A")])

('BA', 'A')

In [31]:
test_string = """low low low low low lower lower widest widest widest newest newest newest newest newest newest"""

In [32]:
re.findall(PAT, test_string)

['low',
 ' low',
 ' low',
 ' low',
 ' low',
 ' lower',
 ' lower',
 ' widest',
 ' widest',
 ' widest',
 ' newest',
 ' newest',
 ' newest',
 ' newest',
 ' newest',
 ' newest']

### 2.5 Experimenting with BPE Tokenizer Training

In [3]:
def find_chunk_boundaries(
    file: BinaryIO,
    desired_num_chunks: int,
    split_special_token: bytes,
) -> list[int]:
    """
    Chunk the file into parts that can be counted independently.
    May return fewer chunks if the boundaries end up overlapping.
    """
    assert isinstance(split_special_token, bytes), "Must represent special token as a bytestring"

    # Get total file size in bytes
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)

    chunk_size = file_size // desired_num_chunks

    # Initial guesses for chunk boundary locations, uniformly spaced
    # Chunks start on previous index, don't include last index
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size

    mini_chunk_size = 4096  # Read ahead by 4k bytes at a time

    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)  # Start at boundary guess
        while True:
            mini_chunk = file.read(mini_chunk_size)  # Read a mini chunk

            # If EOF, this boundary should be at the end of the file
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break

            # Find the special token in the mini chunk
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = initial_position + found_at
                break
            initial_position += mini_chunk_size

    # Make sure all boundaries are unique, but might be fewer than desired_num_chunks
    return sorted(set(chunk_boundaries))

def update_token_freqs(
    token_freqs: dict[TOKEN, int],
    text_segment: str,
):
    matches = re.finditer(PAT, text_segment)
    for m in matches:
        token = m.group()
        token_bytes = tuple(bytes([b]) for b in token.encode("utf-8"))
        token_freqs[token_bytes] += 1 

def process_chunk(args):
    input_path, start, end, special_tokens, chunk_id = args
    # Read chunk
    with open(input_path, 'rb') as f:
        f.seek(start)
        chunk_bytes = f.read(end - start)
    
    # Decode
    chunk_str = chunk_bytes.decode("utf-8", errors="ignore")
    
    # Logic from original train_bpe
    special_pat = "|".join(re.escape(st) for st in special_tokens)
    segments = re.split(special_pat, chunk_str)
    
    token_freqs = defaultdict(int)
    # Use position=chunk_id+1 so 0 is left for the main bar
    for seg in segments:
        update_token_freqs(token_freqs, seg)
    
    return token_freqs

def get_pairs(
    token: TOKEN
) -> list[PAIR]:
    if len(token) < 2:
        return []
    return [(token[i], token[i+1]) for i in range(len(token)-1)]

def update_pair_freqs(
    pair_freqs: dict[PAIR, int],
    token_freqs: dict[TOKEN, int],
):
    for token, freq in token_freqs.items():
        if len(token) < 2:
            continue
        pairs = get_pairs(token)
        for p in pairs:
            pair_freqs[p] += freq

def update_pair2idx(
    pair_to_idx: dict[PAIR, set[TOKEN]],
    token_freqs: dict[TOKEN, int],
    token_to_idx: dict[TOKEN, int],
):
    for token, freq in token_freqs.items():
        if len(token) < 2:
            continue
        pairs = get_pairs(token)
        for p in pairs:
            pair_to_idx[p].add(token_to_idx[token])


def get_most_frequent_pair(
    pair_freqs: dict[PAIR, int],
) -> PAIR:
    return max(pair_freqs.keys(), key=lambda k: (pair_freqs[k], k))

def update_vocab(
    vocab: dict[int, bytes],
    new_id: int,
    best_pair: PAIR,
):
    new_vocab = best_pair[0] + best_pair[1]
    vocab[new_id] = new_vocab

def update_vocab_inverse(
    vocab_inverse: dict[bytes, int],
    new_id: int,
    best_pair: PAIR,
):
    new_vocab = best_pair[0] + best_pair[1]
    vocab_inverse[new_vocab] = new_id

def update_merges(
    merges: list[PAIR],
    best_pair: PAIR,
):
    merges.append(best_pair)

def update_all(
    pair_to_idx: dict[PAIR, set[int]],
    pair_freqs: dict[PAIR, int],
    token_freqs: dict[TOKEN, int],
    idx_to_token: dict[int, TOKEN],
    best_pair: PAIR,
):
    affected_idxs = list(pair_to_idx[best_pair])
    merged_bytes = best_pair[0] + best_pair[1]

    for idx in affected_idxs:
        # get new token
        i=0
        token = idx_to_token[idx]
        new_token = []
        while(i<(len(token))):
            if (i < len(token) - 1) and (token[i] == best_pair[0]) and (token[i+1] == best_pair[1]):
                new_token.append(merged_bytes)
                i = i + 2
            else:
                new_token.append(token[i])
                i = i + 1
        new_token = tuple(new_token)


        ## update pair_to_idx
        new_pairs = get_pairs(new_token)
        affected_pairs = get_pairs(token)
        for p in set(new_pairs)-set(affected_pairs):
            pair_to_idx[p].add(idx)
        for p in set(affected_pairs)-set(new_pairs):
            pair_to_idx[p].discard(idx)


        ## update pair_freqs
        for pair in affected_pairs:
            pair_freqs[pair] -= token_freqs[token]
        for pair in new_pairs:
            pair_freqs[pair] += token_freqs[token]


        ## update token_freqs
        origin_freq = token_freqs.pop(token)
        token_freqs[new_token] = origin_freq

        ## update idx_to_token
        idx_to_token[idx] = new_token
        

def train_bpe(
    input_path: str,
    vocab_size: int,
    special_tokens: list[str],
    num_processes: int = 4
):
    print("Start Training BPE...")
    start_total = time.time()

    print("Initialize Vocab and Merges...")
    vocab = {i:bytes([i]) for i in range(256)}
    vocab_inverse = {v:k for k,v in vocab.items()}
    # Add special tokens
    for st in special_tokens:
        new_id = len(vocab)
        st_bytes = st.encode("utf-8")
        vocab[new_id] = st_bytes
        vocab_inverse[st_bytes] = new_id
    
    merges = []

    print("Calculating chunk boundaries...")
    t0 = time.time()
    with open(input_path, 'rb') as f:
        # Assuming first special token is the split token
        split_token = special_tokens[0].encode("utf-8") if special_tokens else b"<|endoftext|>"
        boundaries = find_chunk_boundaries(f, num_processes, split_token) # More chunks than processes for load balancing
    print(f"Boundaries calculated in {time.time() - t0:.2f}s")
    
    token_freqs = defaultdict(int)

    ## Initiate token_freqs
    print("Update Token Freq (Parallel)...")
    t0 = time.time()
    
    tasks = []
    for i in range(len(boundaries) - 1):
        start = boundaries[i]
        end = boundaries[i+1]
        tasks.append((input_path, start, end, special_tokens, i))
    
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        results = list(tqdm(executor.map(process_chunk, tasks), total=len(tasks), desc="Token Freqs (Chunks)", position=0))
        
        for local_freqs in results:
            for token, count in local_freqs.items():
                token_freqs[token] += count
                
    idx_to_token = {i: k for i, k in enumerate(token_freqs.keys())}
    token_to_idx = {k: i for i, k in enumerate(token_freqs.keys())}
    print(f"Update Token Freq took {time.time() - t0:.2f}s")


    ## Initiate pair_freqs, pair_to_token, and token_to_pair
    print("Update Pair Freq...")
    t0 = time.time()
    pair_freqs = defaultdict(int)
    update_pair_freqs(pair_freqs, token_freqs)
    print(f"Update Pair Freq took {time.time() - t0:.2f}s")

    print("Update Pair to idx...")
    t0 = time.time()
    pair_to_idx = defaultdict(set)
    update_pair2idx(pair_to_idx, token_freqs, token_to_idx)
    print(f"Update Pair to Token took {time.time() - t0:.2f}s")

    print("Start Merging...")
    t0 = time.time()
    num_merges = vocab_size - 256 - len(special_tokens)
    for i in tqdm(range(num_merges), desc="Merging"):
        ### find most frequent pair
        if not pair_freqs:
            break
        best_pair = get_most_frequent_pair(pair_freqs)

        new_id = len(vocab)

        ## update vocab
        update_vocab(vocab, new_id, best_pair)

        ## update vocab_inverse
        update_vocab_inverse(vocab_inverse, new_id, best_pair)

        ## update merges
        update_merges(merges, best_pair)

        update_all(pair_to_idx, pair_freqs, token_freqs, idx_to_token, best_pair)
    print(f"Merging took {time.time() - t0:.2f}s")
    print(f"Total Training took {time.time() - start_total:.2f}s")

    return vocab, merges

#### Prepare

In [ ]:
test_string = "abere ererea<|endoftext|>When and where is not as important as who and what. Hi, I am the the Ivan.<|endoftext|> aaa"
# test_string = "abere ererea"
input_path = ""
vocab_size = 260
special_tokens = [END_TOKEN]

In [5]:
special_pat = "|".join(re.escape(st) for st in special_tokens)
segments = re.split(special_pat, test_string)

text_segment = segments[0]
text_segment

'abere ererea'

#### Initialize Vocab and Merges

In [6]:
vocab = {i:bytes([i]) for i in range(256)}
vocab_inverse = {v:k for k,v in vocab.items()}
# Add special tokens
for st in special_tokens:
    new_id = len(vocab)
    st_bytes = st.encode("utf-8")
    vocab[new_id] = st_bytes
    vocab_inverse[st_bytes] = new_id

merges = []

#### Update Token Freq

In [7]:
token_freqs = defaultdict(int)
update_token_freqs(token_freqs,text_segment)
token_freqs

defaultdict(int,
            {(b'a', b'b', b'e', b'r', b'e'): 1,
             (b' ', b'e', b'r', b'e', b'r', b'e', b'a'): 1})

#### Update idx freq

In [16]:
idx_to_token = {i: k for i, k in enumerate(token_freqs.keys())}
token_to_idx = {k: i for i, k in enumerate(token_freqs.keys())}
idx_freqs = {token_to_idx[token]: freq for token, freq in token_freqs.items()}

#### Update Pair Freq

In [9]:
pair_freqs = defaultdict(int)
update_pair_freqs(pair_freqs,token_freqs)
pair_freqs

defaultdict(int,
            {(b'a', b'b'): 1,
             (b'b', b'e'): 1,
             (b'e', b'r'): 3,
             (b'r', b'e'): 3,
             (b' ', b'e'): 1,
             (b'e', b'a'): 1})

#### Update Pair to idx

In [10]:
pair_to_idx = defaultdict(set)
update_pair2idx(pair_to_idx, token_freqs, token_to_idx)
pair_to_idx

defaultdict(set,
            {(b'a', b'b'): {0},
             (b'b', b'e'): {0},
             (b'e', b'r'): {0, 1},
             (b'r', b'e'): {0, 1},
             (b' ', b'e'): {1},
             (b'e', b'a'): {1}})

#### Merging

In [11]:
best_pair = get_most_frequent_pair(pair_freqs)
best_pair

(b'r', b'e')

In [12]:
new_id = len(vocab)

## update vocab
update_vocab(vocab, new_id, best_pair)

## update vocab_inverse
update_vocab_inverse(vocab_inverse, new_id, best_pair)

## update merges
update_merges(merges, best_pair)

In [17]:
affected_idxs = list(pair_to_idx[best_pair])
merged_bytes = best_pair[0] + best_pair[1]
for idx in affected_idxs:
    token = idx_to_token[idx]

    i=0
    new_token = []
    while(i<(len(token))):
        if (i < len(token) - 1) and (token[i] == best_pair[0]) and (token[i+1] == best_pair[1]):
            new_token.append(merged_bytes)
            i = i + 2
        else:
            new_token.append(token[i])
            i = i + 1
    new_token = tuple(new_token)

    new_pairs = get_pairs(new_token)
    affected_pairs = get_pairs(token)
    for p in set(new_pairs)-set(affected_pairs):
        pair_to_idx[p].add(idx)
    for p in set(affected_pairs)-set(new_pairs):
        pair_to_idx[p].discard(idx)


    for pair in affected_pairs:
        pair_freqs[pair] -= idx_freqs[idx]
    for pair in new_pairs:
        pair_freqs[pair] += idx_freqs[idx]

    # origin_freq = token_freqs.pop(token)
    # token_freqs[new_token] = origin_freq

    idx_to_token[idx] = new_token

In [18]:
pair_to_idx

defaultdict(set,
            {(b'a', b'b'): {0},
             (b'b', b'e'): {0},
             (b'e', b'r'): set(),
             (b'r', b'e'): set(),
             (b' ', b'e'): {1},
             (b'e', b'a'): set(),
             (b'e', b're'): {0, 1},
             (b're', b'a'): {1},
             (b're', b're'): {1}})

In [19]:
pair_freqs

defaultdict(int,
            {(b'a', b'b'): 1,
             (b'b', b'e'): 1,
             (b'e', b'r'): 0,
             (b'r', b'e'): 0,
             (b' ', b'e'): 1,
             (b'e', b'a'): 0,
             (b'e', b're'): 2,
             (b're', b're'): 1,
             (b're', b'a'): 1})

In [86]:
idx_freqs

{0: 1, 1: 1}

In [46]:
idx_to_token

{0: (b'a', b'b', b'e', b're'), 1: (b' ', b'e', b're', b're', b'a')}

### 2.6 BPE Tokenizer: Encoding and Decoding

##### 2.6.1 Encoding text

##### 2.6.2 Decoding text

In [5]:
test_string = "abere ererea<|endoftext|>When and where is not as important as who and what. Hi, I am the the Ivan.<|endoftext|> aaa"

special_tokens = [END_TOKEN]
vocab_filepath = f"/Users/yifanli/Github/stanford-cs336/stanford-cs336-assignment1-basics/outputs/TinyStoriesV2-GPT4-train-vocab-10000.pkl"
merges_filepath = f"/Users/yifanli/Github/stanford-cs336/stanford-cs336-assignment1-basics/outputs/TinyStoriesV2-GPT4-train-merge-10000.pkl"

In [6]:
with open(vocab_filepath, 'rb') as f:
    vocab = pickle.load(f)
with open(merges_filepath, 'rb') as f:
    merges = pickle.load(f)

In [22]:
special_pat = "|".join(re.escape(st) for st in special_tokens)
segments = re.split(special_pat, test_string)

text_segment = segments[0]
text_segment

'abere ererea'

In [75]:
token = ' ererea'
token_bytes = list(bytes([b]) for b in token.encode("utf-8"))
token_bytes

[b' ', b'e', b'r', b'e', b'r', b'e', b'a']

In [76]:
while 1:
    merge_position = -1
    smallest_rank = len(merges)
    for i in range(len(token_bytes)-1):
        current_pair = (token_bytes[i], token_bytes[i+1])
        rank = merges_to_rank.get(current_pair, -1)
        if rank == -1:
            continue
        if rank < smallest_rank:
            smallest_rank = rank
            merge_position = i
    if merge_position == -1:
        break
    print(token_bytes[merge_position], token_bytes[merge_position+1])
    token_bytes = token_bytes[:(merge_position)] + [token_bytes[merge_position]+token_bytes[merge_position+1]]+token_bytes[(merge_position+2):]


b'r' b'e'
b'r' b'e'
b' ' b'e'


In [87]:
encoded_token = []
for i in token_bytes:
    encoded_token.append(vocab_inverse.get(i, "aaa"))
np.array(encoded_token, dtype=np.uint16)

array([345, 273, 273,  97], dtype=uint16)

In [35]:
class Tokenizer():
    def __init__(
        self, 
        vocab: dict[int, bytes], 
        merges: list[PAIR],
        special_tokens: list[str] | None = None,
    ):
        self.vocab = vocab
        self.merges = merges

        if special_tokens:
            new_index = len(vocab)
            for st in special_tokens:
                vocab[new_index] = st
                new_index += 1
            self.special_tokens =  sorted([END_TOKEN]+special_tokens, key=len, reverse=True)
        else:
            self.special_tokens = [END_TOKEN]
        
        self.vocab_inverse = {v: k for k, v in vocab.items()}
        self.merges_to_rank = {m: i for i, m in enumerate(merges)}


    @classmethod
    def from_files(
        cls, 
        vocab_filepath: str, 
        merges_filepath: str, 
        special_tokens: list[str] | None = None,
    ) :
        with open(vocab_filepath, "rb") as f:
            vocab = pickle.load(f)
        
        with open(merges_filepath, "rb") as f:
            merges = pickle.load(f)
        return cls(vocab, merges, special_tokens=special_tokens)


    def merge_tokens(self, token_bytes: list[bytes]) -> list[bytes]:
        while 1:
            merge_position = -1
            smallest_rank = len(self.merges)
            for i in range(len(token_bytes)-1):
                current_pair = (token_bytes[i], token_bytes[i+1])
                rank = self.merges_to_rank.get(current_pair, -1)
                if rank == -1:
                    continue
                if rank < smallest_rank:
                    smallest_rank = rank
                    merge_position = i
            if merge_position == -1:
                break
            token_bytes = token_bytes[:(merge_position)] + [token_bytes[merge_position]+token_bytes[merge_position+1]]+token_bytes[(merge_position+2):]
        return token_bytes

    def encode(self, text: str) -> list[int]:
        token_bytes = list(bytes([b]) for b in text.encode("utf-8"))
        token_bytes = self.merge_tokens(token_bytes)
        encoded_token = []
        for i in token_bytes:
            encoded_token.append(self.vocab_inverse.get(i, "aaa"))
        return encoded_token

    def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]:
        pass

    def decode(self, ids: list[int]) -> str:
        unk_bytes = '\ufffd'.encode('utf-8')
        bytes_list = [self.vocab.get(i, unk_bytes) for i in ids]
        return b"".join(bytes_list).decode("utf-8", errors="replace")


In [36]:
tokenizer = Tokenizer(vocab, merges)

In [37]:
tokenizer = Tokenizer.from_files(vocab_filepath=vocab_filepath, merges_filepath=merges_filepath)

In [38]:
ids = tokenizer.encode('Hello, world!')
ids

[1183, 44, 1569, 33]

In [39]:
segment = tokenizer.decode(ids)
segment

'Hello, world!'